<a href="https://colab.research.google.com/github/binram077/SQL-random-data-generator/blob/main/public_SQLdata_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install names

     |████████████████████████████████| 789 kB 5.0 MB/s 
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803698 sha256=60f0028f8a92bdd515beb649d80ebd447de9ca01f6253de3ad78215bacaf79e5
  Stored in directory: /root/.cache/pip/wheels/05/ea/68/92f6b0669e478af9b7c3c524520d03050089e034edcc775c2b
Successfully built names


# sub-generators (values generation functions)
 (e.g. a generator for Ints between (a,b) without duplicate values)

In [ ]:
import names
import copy
import datetime
import random

def randomStringsWithoutDuplicateValues(length,numberOfStrings):
  shuffled = []
  for i in range(0,length,5):
    shuffled.append(0)
  for idx,i in enumerate(range(0,length,5)):
    if i+5 < length:
      shuffled[idx] = list(range(100000))
      random.shuffle(shuffled[idx])
      shuffled[idx] = shuffled[idx][:numberOfStrings]
    else:
      shuffled[idx] = list(range(10 ** (length - i - 1)))
      random.shuffle(shuffled[idx])
      shuffled[idx] = shuffled[idx][:numberOfStrings]
  res = []
  for i in range(numberOfStrings):
    element = ""
    for idx,j in enumerate(range(0,length,5)):
      if j+5 < length:
        element += str(shuffled[idx][i]).zfill(5)
      else:
        element += str(shuffled[idx][i]).zfill(length - j)
    res.append("'" + element + "'")
  return res

def VarCharIDs(length,numberOfStrings):
  return randomStringsWithoutDuplicateValues(length,numberOfStrings)

def RandomNames(_,numberOfStrings):
  NamesArr =[]
  for i in range(numberOfStrings):
    NamesArr.append("'"+names.get_full_name()+"'")
  return NamesArr

def RandomFromList(ListOfOptions,numberOfObjects,inheritence = False):
  ResArr = []
  for i in range(numberOfObjects):
    idx = random.randint(0,len(ListOfOptions)-1)
    if inheritence:
      ResArr.append(ListOfOptions[idx])
    else:
      ResArr.append("'" + ListOfOptions[idx] + "'")
  return ResArr

def RandomFromListWD(ListOfOptions,numberOfObjects,inheritence = False):
  ShuffledOptions = (copy.deepcopy(ListOfOptions))
  random.shuffle(ShuffledOptions)
  if not inheritence:
    for x in ShuffledOptions:
      x = "'" + x + "'"
  return ShuffledOptions[:numberOfObjects]

def random_date(start, end):
    """Generate a random datetime between `start` and `end`"""
    return start + datetime.timedelta(
        # Get a random amount of seconds between `start` and `end`
        seconds=random.randint(0, int((end - start).total_seconds())),
    )

def RandomDates(_,numberOfDates):
  ResArr = []
  for i in range(numberOfDates):
    date = random_date(datetime.datetime(2020, 1, 1),datetime.datetime(2030, 1, 1))
    ResArr.append("%s-%s-%s" % (date.year,date.month,date.day))
  return ResArr

def RandomInts(range_of_nums,numberOfInts):
  ResArr = []
  for i in range(numberOfInts):
    ResArr.append(str(random.randint(range_of_nums[0],range_of_nums[1])))
  return ResArr

def IntsWD(range_of_nums,numberOfInts):
  ResArr = range(range_of_nums[0],range_of_nums[1])
  random.shuffle(ResArr)
  return ResArr[:numberOfInts]

def InheritVarCharIDs(ListOfOptions,numberOfObjects):
  return RandomFromList(ListOfOptions,numberOfObjects,True)


# "Table" class, that is used to handle the data.

In [ ]:
class Table():
  Dictionary = {'ID_Var' : VarCharIDs,
                'ID_Var_key' : VarCharIDs,
                'Inherited_ID_Var' : InheritVarCharIDs,
                'Name' : RandomNames,
                'FromList' : RandomFromList,
                'FromListWithoutDup' : RandomFromListWD,
                'Date': RandomDates,
                'Ints' : RandomInts,
                'IntsWithoutDup' : IntsWD,
                'preprocessed' : None,}
  def __init__(self,_Name,_Tags = None,_Types_and_Ranges = None,_DataLength = 10):
    #print(_Name)
    self.Name = _Name
    self.Tags = _Tags
    if _Tags ==None:
      self.Tags = []
    #print(self.Tags)
    self.Types_and_Ranges = _Types_and_Ranges
    if _Types_and_Ranges ==None:
      self.Types_and_Ranges = []
    self.raws = []
    self.DataLength = _DataLength
    self.keys = {'tags' : [], 'vals' : []}
    for idx,_ in enumerate(self.Types_and_Ranges):
      if _Types_and_Ranges[idx][0] == "ID_Var_key":
        self.keys['tags'].append(self.Tags[idx])
        self.Tags.remove(self.Tags[idx])
  
  def switch(self,x):
    return self.Dictionary.get(x)

  def create_raws(self):
    raws = []
    for T_R in self.Types_and_Ranges:
      if T_R[0] != "ID_Var_key":
        self.raws.append(self.switch(T_R[0])(T_R[1],self.DataLength))
      else:
        self.keys['vals'].append(self.switch(T_R[0])(T_R[1],self.DataLength))

  def inherit(self, DataTable):
    for idx,_ in enumerate(DataTable.keys['tags']):
      self.keys['tags'].append(DataTable.keys['tags'][idx])
      self.keys['vals'].append(RandomFromList(DataTable.keys['vals'][idx],self.DataLength, inheritence = True))
      print(self.keys['tags'])
      


# Implementation of ratios and convertion function from "table" class to SQL Insert commands.

In [ ]:
def RatioTwoTables(Name = "ratio",Tables = [],_Tags = [],_Types_and_Ranges = [],DataLength = 0,Many = False,ToMany = False):
  result = Table(Name,_Tags,_Types_and_Ranges,DataLength)
  result.create_raws()
  keysTagsList = Tables[0].keys['tags'] + Tables[1].keys['tags']
  keysValsList = [RandomFromList(x,DataLength,True) if Many else RandomFromListWD(x,DataLength,True) for x in Tables[0].keys['vals']] + [RandomFromList(x,DataLength,True) if ToMany else RandomFromListWD(x,DataLength,True) for x in Tables[1].keys['vals']]
  #print(keysValsList)
  #print("aaa",[x for x in Tables[0].keys['vals']])
  for i in range(len(keysTagsList)):
    result.keys['tags'].append(keysTagsList[i])
    result.keys['vals'].append(keysValsList[i])
    #print(keysValsList[i])
  return result

def ConvertValuesToSQL_commands(DataTable):
  commands = []
  for i in range(DataTable.DataLength):
    raw = []
    for j,_ in enumerate(DataTable.keys['tags']):
      #print(DataTable.keys['vals'][j])
      z = DataTable.keys['vals'][j]
      raw.append(DataTable.keys['vals'][j][i])
    for j,_ in enumerate(DataTable.Tags):
      raw.append(DataTable.raws[j][i])
    #print(DataTable.Name)
    #print(DataTable.keys['tags'] + DataTable.Tags)
    #print(raw)
    #print(','.join(raw))
    commands.append("INSERT INTO %s (%s) VALUES (%s);" % (DataTable.Name,", ".join(DataTable.keys['tags'] + DataTable.Tags), ", ".join(raw)))
  return "\n".join(commands)



def FileWriteTable(DataTable):
  text_file = open(DataTable.Name + ".txt", "w")

  text_file.write(ConvertValuesToSQL_commands(DataTable))

  text_file.close()

CREATE TABLE department
(
  DeptID VARCHAR(10) NOT NULL,
  HeadName VARCHAR(15) NOT NULL,
  HeadID VARCHAR(9) NOT NULL,
  PRIMARY KEY (DeptID)
);

CREATE TABLE Student
(
  StudentID VARCHAR(9) NOT NULL,
  Name VARCHAR(20) NOT NULL,
  PRIMARY KEY (StudentID)
);

CREATE TABLE Course
(
  CourseID VARCHAR(10) NOT NULL,
  name VARCHAR(15) NOT NULL,
  CoordinatorName VARCHAR(15) NOT NULL,
  numberOfPoints INT NOT NULL,
  minimumGrade INT NOT NULL,
  Required CHAR(1) NOT NULL,
  CoordID VARCHAR(10) NOT NULL,
  DeptID VARCHAR(10) NOT NULL,
  PRIMARY KEY (CourseID, DeptID),
  FOREIGN KEY (DeptID) REFERENCES department(DeptID)
);

CREATE TABLE Test
(
  TestID VARCHAR(10) NOT NULL,
  ExamTime CHAR(1) NOT NULL,
  Date DATE NOT NULL,
  ExamLength INT NOT NULL,
  Reference VARCHAR(50) NOT NULL,
  CourseID VARCHAR(10) NOT NULL,
  DeptID VARCHAR(10) NOT NULL,
  PRIMARY KEY (TestID, CourseID, DeptID),
  FOREIGN KEY (CourseID, DeptID) REFERENCES Course(CourseID, DeptID)
);

CREATE TABLE studentResults
(
  Grade INT NOT NULL,
  passed CHAR(1) NOT NULL,
  StudentID VARCHAR(9) NOT NULL,
  TestID VARCHAR(10) NOT NULL,
  CourseID VARCHAR(10) NOT NULL,
  DeptID VARCHAR(10) NOT NULL,
  PRIMARY KEY (StudentID, TestID, CourseID, DeptID),
  FOREIGN KEY (StudentID) REFERENCES Student(StudentID),
  FOREIGN KEY (TestID, CourseID, DeptID) REFERENCES Test(TestID, CourseID, DeptID)
);

CREATE TABLE Prerequisite_to
(
  CourseID_1 VARCHAR(10) NOT NULL,
  DeptID VARCHAR(10) NOT NULL,
  Prerequisite_toCourseID_2 VARCHAR(10) NOT NULL,
  Prerequisite_toDeptID VARCHAR(10) NOT NULL,
  PRIMARY KEY (CourseID_1, DeptID, Prerequisite_toCourseID_2, Prerequisite_toDeptID),
  FOREIGN KEY (CourseID_1, DeptID) REFERENCES Course(CourseID, DeptID),
  FOREIGN KEY (Prerequisite_toCourseID_2, Prerequisite_toDeptID) REFERENCES Course(CourseID, DeptID)
);

CREATE TABLE Appeal
(
  AppID VARCHAR(10) NOT NULL,
  accepted CHAR(1) NOT NULL,
  points INT NOT NULL,
  StudentID VARCHAR(9) NOT NULL,
  TestID VARCHAR(10) NOT NULL,
  CourseID VARCHAR(10) NOT NULL,
  DeptID VARCHAR(10) NOT NULL,
  PRIMARY KEY (AppID, StudentID, TestID, CourseID, DeptID),
  FOREIGN KEY (StudentID, TestID, CourseID, DeptID) REFERENCES studentResults(StudentID, TestID, CourseID, DeptID)
);

CREATE TABLE regitered
(
  StudentID VARCHAR(9) NOT NULL,
  TestID VARCHAR(10) NOT NULL,
  CourseID VARCHAR(10) NOT NULL,
  DeptID VARCHAR(10) NOT NULL,
  PRIMARY KEY (StudentID, TestID, CourseID, DeptID),
  FOREIGN KEY (StudentID) REFERENCES Student(StudentID),
  FOREIGN KEY (TestID, CourseID, DeptID) REFERENCES Test(TestID, CourseID, DeptID)
);

# Initialize table with .csv file

In [ ]:
import pandas as pd

def CSVtoTable(path,keyNames,Name):
  df = pd.read_csv(path)
  res = Table(Name)
  res.__init__(Name)
  nameOfCols = df.columns
  ColsContent = df.to_dict('list')
  for name in nameOfCols:
    if name in keyNames:
      res.keys['tags'].append(name)
      res.keys['vals'].append(["'"+str(x)+"'" for x in ColsContent[name]])
      res.Types_and_Ranges.append(["ID_Var_key",len(ColsContent[name])])
    if name not in keyNames:
      res.Tags.append(name)
      res.raws.append(["'"+str(x)+"'" for x in ColsContent[name]])
      res.Types_and_Ranges.append(["preprocessed",len(ColsContent[name])])
  return res

# Use case example

In [ ]:
#Initalize a table with the name Student, and columns names : "StudentID" and "Name"
#after that we define what values we should generate for each column, in our case:
#"StudentID" is a varchar key with length of 9, and "Name" is surprisingly defined as "Name" in order to call the names generator.
#finally we define the number of raws that we will generate. 
Student = Table(_Name = "Student",_Tags = ["StudentID","Name"],_Types_and_Ranges = [('ID_Var_key',9),('Name',_)],_DataLength = 150)
# after the setting we call the "create_raws" function in order to generate the raws respectively.
Student.create_raws()

#same as student
Department = Table(_Name = "Department",_Tags = ["DeptID","HeadName","HeadID"],_Types_and_Ranges = [('ID_Var_key',10),('Name',_),('ID_Var',9)],_DataLength = 4)
Department.create_raws()

#two differences: 
#1. we use the from_list generator in order to get the course_name which get a predefined list
#2. we use the inheritence function because Course identity is depends on his deparment (infi for computer science is different from infi for math)
CourseNames = ["mavo","sifratiot","infi1","infi2","algebra1","algebra2","algorithms1","algorithms2","DBs","MP in DBs","advanced c++","OSs","Automats","compilers"]
Course = Table(_Name = "Course",_Tags = ["CourseID","name","CoordinatorName","numberOfPoints","minimumGrade","Required","CoordID"],_Types_and_Ranges = [('ID_Var_key',10),('FromListWithoutDup',CourseNames),('Name',_),('Ints',[1,5]),('Ints',[55,80]),('FromList',["Y","N"]),('ID_Var',10)],_DataLength = 12)
Course.create_raws()
Course.inherit(Department)

#same
Test = Table(_Name = "Test",_Tags = ["TestID","ExamTime","Date","ExamLength","Reference"],_Types_and_Ranges = [('ID_Var_key',10),('FromList',["A","B","S"]),('Date',_),('Ints',[90,180]),('FromList',["Calculator","A4 page","the material is on the moodle"])],_DataLength = 23)
Test.create_raws()
Test.inherit(Course)

#same, note that multiple inheritence is possible
studentResults = Table(_Name = "studentResults",_Tags = ["Grade","passed"],_Types_and_Ranges = [('Ints',[90,180]),('Name',_)],_DataLength = 150)
studentResults.create_raws()
studentResults.inherit(Test)
studentResults.inherit(Student)

# learn uses the "RatioTwoTables" in order to create a ratio between Course and student
learn = RatioTwoTables(Name='CourseStudent',Tables=[Course,Student],DataLength = 10)

#write the table to SQL's insert commands in a text file
FileWriteTable(learn)
FileWriteTable(Student)
FileWriteTable(Department)
FileWriteTable(Course)
FileWriteTable(Test)
FileWriteTable(studentResults)

['CourseID', 'DeptID']
['TestID', 'CourseID']
['TestID', 'CourseID', 'DeptID']
['TestID']
['TestID', 'CourseID']
['TestID', 'CourseID', 'DeptID']
['TestID', 'CourseID', 'DeptID', 'StudentID']
